## This script is used to compare the results from all 3 software.
## Written by Yusreen Shah
## Date: May 10th 2023

In [1]:
# import the libraries
import numpy as np
import pandas as pd
import re 
from Bio import SeqIO
from collections import defaultdict


## This section saves the list of all the queries in a dataframe

In [2]:
# Save the data
data = defaultdict(list)

# Read the values from the .fasta file, and save them to data
for seq_record in SeqIO.parse("combined_1.fasta", "fasta"):
    query=seq_record.id
    sequence= repr(seq_record.seq)
    length=len(seq_record)
    data['Query'].append(query)
    data['Sequence'].append(sequence)
    data['Length'].append(length)
    
# Add the data to a dataframe
df = pd.DataFrame.from_dict(data)


In [3]:
# Remove Seq(' and ') from the sequences
df['Sequence'] = df['Sequence'].str.replace('Seq(''', '')
df['Sequence'] = df['Sequence'].str.replace(')', '')
df['Sequence'] = df['Sequence'].str.strip(" \' ")
df['Sequence']=df['Sequence'].str.rstrip()

In [4]:
df=df.sort_values("Query")
df=df.reset_index(drop=True)
df

,Query,Sequence,Length
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861
...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858


## This section is used to compare the results from Bandage and the actual list of queries.

In [5]:
# Create a dataframe for Bandage Combined1
Bandage_Combined1= pd.read_csv('Bandageoutputcombined1.tsv', sep='\t')

In [6]:
Bandage_Combined1.head()

,Query,Path,Length,Query covered by path,Query covered by hits,Mean hit identity,Total hit mismatches,Total hit gap opens,Relative length,Length discrepancy,E-value product,Sequence
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,(56) 7593+ (642),587,95.915%,95.915%,99.83%,1,0,100%,0,0,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...
1,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,(93) 7593+ (642),550,96.4912%,96.4912%,99.455%,3,0,100%,0,0,CAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGCATGAC...
2,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,(88) 7593+ (642),555,100%,100%,99.64%,2,0,100%,0,0,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...
3,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,(88) 7593+ (642),555,100%,100%,99.64%,2,0,100%,0,0,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...
4,gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,(56) 7593+ (642),587,95.915%,95.915%,99.659%,2,0,100%,0,0,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...


In [7]:
Bandage_Combined_Query_Path =Bandage_Combined1[['Path','Query','Sequence']]

## Extract the start and end position from the path in Bandage

In [8]:
# Extract the start position
pattern_path_start = r'\((.*?)\)' 

In [9]:
Bandage_Combined_Query_Path['Start']=Bandage_Combined_Query_Path['Path'].str.extract(pattern_path_start, expand=False)
Bandage_Combined_Query_Path

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1467/3300280694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['Start']=Bandage_Combined_Query_Path['Path'].str.extract(pattern_path_start, expand=False)


,Path,Query,Sequence,Start
0,(56) 7593+ (642),gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...,56
1,(93) 7593+ (642),gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,CAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGCATGAC...,93
2,(88) 7593+ (642),gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...,88
3,(88) 7593+ (642),gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...,88
4,(56) 7593+ (642),gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...,56
...,...,...,...,...
388,(37) 5967+ (897),gb|AJ318094|+|0-861|ARO:3000961|TEM-94,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37
389,(37) 5967+ (897),gb|AJ308558|+|181-1042|ARO:3000962|TEM-95,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37
390,(37) 5967+ (897),gb|AY092401|+|0-861|ARO:3000963|TEM-96,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37
391,(33708) 1107- (35627),gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM,ATGAAAATTATTAATATTGGAGTTTTAGCTCATGTTGATGCAGGAA...,33708


In [10]:
# Extract the end position
Bandage_Combined_Query_Path['End']= Bandage_Combined_Query_Path["Path"].str.split().str[-1]
Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('()').astype(int)
Bandage_Combined_Query_Path

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1467/1842257949.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['End']= Bandage_Combined_Query_Path["Path"].str.split().str[-1]
/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1467/1842257949.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bandage_Combined_Query_Path['End']=Bandage_Combined_Query_Path['End'].str.strip('()').astype(int)


,Path,Query,Sequence,Start,End
0,(56) 7593+ (642),gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...,56,642
1,(93) 7593+ (642),gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,CAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGCATGAC...,93,642
2,(88) 7593+ (642),gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...,88,642
3,(88) 7593+ (642),gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,GTGACCAACAGCAACGATTCCGTCACACTGCGCCTCATGACTGAGC...,88,642
4,(56) 7593+ (642),gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,AAACAAAGTTAGGCATCACAAAGTACAGCATCGTGACCAACAGCAA...,56,642
...,...,...,...,...,...
388,(37) 5967+ (897),gb|AJ318094|+|0-861|ARO:3000961|TEM-94,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,897
389,(37) 5967+ (897),gb|AJ308558|+|181-1042|ARO:3000962|TEM-95,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,897
390,(37) 5967+ (897),gb|AY092401|+|0-861|ARO:3000963|TEM-96,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,37,897
391,(33708) 1107- (35627),gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM,ATGAAAATTATTAATATTGGAGTTTTAGCTCATGTTGATGCAGGAA...,33708,35627


In [11]:
# Reorder the columns in Bandage_Combined_Query_Path
Bandage_Combined_Query_Path=Bandage_Combined_Query_Path[['Query','Path','Start','End','Sequence']]

# Convert integer columns to strings
Bandage_Combined_Query_Path['Path'] = Bandage_Combined_Query_Path['Path'].astype(str)
Bandage_Combined_Query_Path['End'] = Bandage_Combined_Query_Path['End'].astype(str)
Bandage_Combined_Query_Path['Start'] = Bandage_Combined_Query_Path['Start'].astype(str)
Bandage_Combined_Query_Path['Sequence'] = Bandage_Combined_Query_Path['Sequence'].astype(str)

# Bandage_Combined1['Query'].value_counts() 
# Merge the rows that have the same query
# group the dataframe by the 'Name' column and aggregate the data for each group
merge_Bandage_df = Bandage_Combined_Query_Path.groupby('Query').agg({'Path': ' , '.join, 'Start': ', '.join, 'End': ', '.join, 'Sequence': ', '.join}).reset_index()
merge_Bandage_df

,Query,Path,Start,End,Sequence
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,(37) 5967+ (897),37,897,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
...,...,...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,(37) 5967+ (897),37,897,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...


In [12]:
# Add Braces to the values that now have ','
def add_braces(value):
    if ',' in value:
        return '{' + value + '}'
    else:
        return value
merge_Bandage_df['Path'] = merge_Bandage_df['Path'].apply(add_braces)
merge_Bandage_df['Start'] = merge_Bandage_df['Start'].apply(add_braces)
merge_Bandage_df['End'] = merge_Bandage_df['End'].apply(add_braces)
merge_Bandage_df['Sequence'] = merge_Bandage_df['Sequence'].apply(add_braces)

In [13]:
# Store the dataframe containing the Bandage results to a .tsv file
merge_Bandage_df.to_csv("Test1.tsv", sep="\t")

In [14]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsBandage=pd.merge(df,merge_Bandage_df, on='Query',how="outer")

## Compile the Queries and the results from Bandage in one dataframe

In [15]:
df_QueryAndResultsBandage

,Query,Sequence_x,Length,Path,Start,End,Sequence_y
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
...,...,...,...,...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...


In [16]:
#for col in df_QueryAndResultsBandage.columns:
   # print(col)

In [17]:
#Print the columns from the dataframe
df_QueryAndResultsBandage.columns

Index(['Query', 'Sequence_x', 'Length', 'Path', 'Start', 'End', 'Sequence_y'], dtype='object')

In [18]:
df_QueryAndResultsBandage.rename(columns = {'Sequence_x':'Sequence'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Sequence_y':'Sequence_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Length':'Length_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Path':'Path_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'Start':'Start_Bandage'}, inplace = True)
df_QueryAndResultsBandage.rename(columns = {'End':'End_Bandage'}, inplace = True)

## Save the Queries and Results from Bandage in one .tsv file

In [19]:
df_QueryAndResultsBandage.to_csv("CompiledQueriesBandage.tsv",sep="\t")

## This section is used to compare the results from SPAligner and the actual list of queries.

In [20]:
#Create a dataframe for SPAligner Combined1
SPAligner_Combined1=pd.read_csv('SPAligneroutputcombined1.tsv', sep='\t')
SPAligner_Combined1

,Query,Start position of alignment on sequence,End position of alignment on sequence,Start position of alignment on the first edge of the path,End position on the last edge of the path,Sequence length,Path of alignment,Length of the alignment on each edge of the path,Sequence
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10 [...,0,612,39,642,612,7593+,603,AGTATTCAACATTTCCAAACAAAGTTAGGCATCACAAAGTACAGCA...
1,gb|AB700703|+|161-1022|ARO:3001057|TEM-198 [Kl...,0,861,36,897,861,5967+,861,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
2,gb|AF535130|+|0-861|ARO:3001100|SHV-42 [Klebsi...,0,861,23,884,861,5151-,861,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
3,gb|EF373972|+|0-861|ARO:3001147|SHV-95 [Citrob...,0,861,23,884,861,5151-,861,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
4,gb|JX121126|+|0-858|ARO:3001198|SHV-159 [Klebs...,0,858,23,881,858,5151-,858,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
...,...,...,...,...,...,...,...,...,...
233,gb|FJ668814|+|76-937|ARO:3001059|SHV-1 [Klebsi...,0,861,23,884,861,5151-,861,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
234,gb|M88143|+|367-1228|ARO:3000884|TEM-12 [Klebs...,0,861,36,897,861,5967+,861,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
235,gb|FR848831|+|0-861|ARO:3001375|TEM-184 [Esche...,0,861,36,897,861,5967+,861,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
236,gb|HM559945|+|0-861|ARO:3001178|SHV-134 [Klebs...,0,861,23,884,861,5151-,861,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...


In [21]:
#get the columns from SPAligner
SPAligner_Combined1.columns

Index(['Query', 'Start position of alignment on sequence',
       'End position of alignment on sequence',
       'Start position of alignment on the first edge of the path',
       'End position on the last edge of the path', 'Sequence length',
       'Path of alignment',
       'Length of the alignment on each edge of the path ', 'Sequence'],
      dtype='object')

In [22]:
#format the Query column from  SPAligner_Combined1 so that the Query column is the same as the one from Bandage
pattern = r'\[.*?\]'
def format_query(x):
    return re.sub(pattern,"", x).rstrip()
    


SPAligner_Combined1['Query'] = SPAligner_Combined1['Query'].map(format_query)


In [23]:
#Get the query and sequence column from SPAligner_Combined1
SPAligner_Combined_Query_Path=SPAligner_Combined1[['Query','Sequence']]


In [24]:
#Get the query and sequence column from SPAligner_Combined1
SPAligner_Combined_Query_Path=SPAligner_Combined1[['Query','Start position of alignment on the first edge of the path', 'End position on the last edge of the path', 'Sequence length',
'Path of alignment','Sequence']]
SPAligner_Combined_Query_Path

,Query,Start position of alignment on the first edge of the path,End position on the last edge of the path,Sequence length,Path of alignment,Sequence
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,39,642,612,7593+,AGTATTCAACATTTCCAAACAAAGTTAGGCATCACAAAGTACAGCA...
1,gb|AB700703|+|161-1022|ARO:3001057|TEM-198,36,897,861,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
2,gb|AF535130|+|0-861|ARO:3001100|SHV-42,23,884,861,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
3,gb|EF373972|+|0-861|ARO:3001147|SHV-95,23,884,861,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
4,gb|JX121126|+|0-858|ARO:3001198|SHV-159,23,881,858,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
...,...,...,...,...,...,...
233,gb|FJ668814|+|76-937|ARO:3001059|SHV-1,23,884,861,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
234,gb|M88143|+|367-1228|ARO:3000884|TEM-12,36,897,861,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
235,gb|FR848831|+|0-861|ARO:3001375|TEM-184,36,897,861,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
236,gb|HM559945|+|0-861|ARO:3001178|SHV-134,23,884,861,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...


In [25]:
#Rename the columns in SPAligner
SPAligner_Combined_Query_Path.rename(columns = {'Start position of alignment on the first edge of the path':'Start_SPAligner'}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'End position on the last edge of the path':'End_SPAligner'}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'Sequence length':"Length_SPAligner"}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'Path of alignment':"Path_SPAligner"}, inplace = True)
SPAligner_Combined_Query_Path.rename(columns = {'Sequence':"Sequence_SPAligner"}, inplace = True)

SPAligner_Combined_Query_Path

/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1467/900970198.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SPAligner_Combined_Query_Path.rename(columns = {'Start position of alignment on the first edge of the path':'Start_SPAligner'}, inplace = True)
/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1467/900970198.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SPAligner_Combined_Query_Path.rename(columns = {'End position on the last edge of the path':'End_SPAligner'}, inplace = True)
/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1467/900970198.py:4: SettingWithCopyWarning: 

,Query,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,39,642,612,7593+,AGTATTCAACATTTCCAAACAAAGTTAGGCATCACAAAGTACAGCA...
1,gb|AB700703|+|161-1022|ARO:3001057|TEM-198,36,897,861,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
2,gb|AF535130|+|0-861|ARO:3001100|SHV-42,23,884,861,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
3,gb|EF373972|+|0-861|ARO:3001147|SHV-95,23,884,861,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
4,gb|JX121126|+|0-858|ARO:3001198|SHV-159,23,881,858,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
...,...,...,...,...,...,...
233,gb|FJ668814|+|76-937|ARO:3001059|SHV-1,23,884,861,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
234,gb|M88143|+|367-1228|ARO:3000884|TEM-12,36,897,861,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
235,gb|FR848831|+|0-861|ARO:3001375|TEM-184,36,897,861,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
236,gb|HM559945|+|0-861|ARO:3001178|SHV-134,23,884,861,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...


## Compile the Queries and the results from SPAligner in one dataframe

In [26]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsSPAligner=pd.merge(df,SPAligner_Combined_Query_Path, on='Query',how='left')
df_QueryAndResultsSPAligner

,Query,Sequence,Length,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,NaN,NaN,NaN,NaN,NaN
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,36.0,897.0,861.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,23.0,884.0,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,23.0,884.0,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,23.0,884.0,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...
...,...,...,...,...,...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,36.0,897.0,861.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,36.0,894.0,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,36.0,894.0,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,36.0,894.0,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...


In [27]:
df_QueryAndResultsSPAligner.rename(columns = {'Sequence_x':'Sequence'}, inplace = True)
df_QueryAndResultsSPAligner.rename(columns = {'Sequence_y':'Sequence_SPAligner'}, inplace = True)

## This section is used to compare the results from GraphAligner and the actual list of queries.

In [28]:
#Create a dataframe for GraphAligner Combined1
GraphAligner_Combined1=pd.read_csv('GraphAligneroutputcombined1.tsv', sep='\t', names=["Query", "Query Length", "Query Start", 
                                        "Query End","Strand Relative Length","Path Matching","Path Length",
                                         "Start Position on Path","End Position on Path","Number of residues Matches",
                                         "Alignment Back Length","Mapping Quality","Column 1"])
GraphAligner_Combined1

,Query,Query Length,Query Start,Query End,Strand Relative Length,Path Matching,Path Length,Start Position on Path,End Position on Path,Number of residues Matches,Alignment Back Length,Mapping Quality,Column 1
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10 [...,612,25,612,+,>7593,2524,55,642,586,587,255,NM:i:1
1,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10 [...,612,2,18,+,<1851,79238,37281,37296,15,16,255,NM:i:1
2,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11...,570,20,570,+,>7593,2524,92,642,547,550,255,NM:i:3
3,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib...,555,0,555,+,>7593,2524,87,642,553,555,255,NM:i:2
4,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib...,555,0,555,+,>7593,2524,87,642,553,555,255,NM:i:2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,gb|AJ318094|+|0-861|ARO:3000961|TEM-94 [Escher...,861,0,861,+,>5967,1044,36,897,852,861,255,NM:i:9
376,gb|AJ308558|+|181-1042|ARO:3000962|TEM-95 [Esc...,861,0,861,+,>5967,1044,36,897,859,861,255,NM:i:2
377,gb|AY092401|+|0-861|ARO:3000963|TEM-96 [Escher...,861,0,861,+,>5967,1044,36,897,858,861,255,NM:i:3
378,gb|AM990992.1|-|1001760-1003680|ARO:3000186|te...,1920,0,1920,+,<1107,47514,33707,35627,1874,1920,255,NM:i:46


In [29]:
#Format the Query column from  GraphAligner_Combined1 so that the Query column is the same as the one from Bandage
pattern = r'\[.*?\]'
def format_query(x):
    return re.sub(pattern,"", x).rstrip()
    


GraphAligner_Combined1['Query'] = GraphAligner_Combined1['Query'].map(format_query)

In [30]:
# Remove < and > from the Path
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].str.replace('>', "")
GraphAligner_Combined1['Path Matching'] = GraphAligner_Combined1['Path Matching'].str.replace('<', "")

## This section drops the rows with coverages < 50 from GraphAligner_Combined1

In [31]:
# Calculate the coverage for GraphAligner
GraphAligner_Combined1["Coverage"]=GraphAligner_Combined1["Alignment Back Length"]/GraphAligner_Combined1["Query Length"]*100

In [32]:
#Drop the coverages that are < 50 from GraphAligner 
GraphAligner_Combined1.drop(GraphAligner_Combined1[GraphAligner_Combined1['Coverage'] < 50].index, inplace = True)
GraphAligner_Combined1

,Query,Query Length,Query Start,Query End,Strand Relative Length,Path Matching,Path Length,Start Position on Path,End Position on Path,Number of residues Matches,Alignment Back Length,Mapping Quality,Column 1,Coverage
0,gb|U59183|+|247-859|ARO:3002581|AAC(6')-Ib10,612,25,612,+,7593,2524,55,642,586,587,255,NM:i:1,95.915033
2,gb|AY136758|+|377-947|ARO:3002582|AAC(6')-Ib11,570,20,570,+,7593,2524,92,642,547,550,255,NM:i:3,96.491228
3,gb|FJ854362|+|1702-2257|ARO:3002576|AAC(6')-Ib3,555,0,555,+,7593,2524,87,642,553,555,255,NM:i:2,100.000000
4,gb|AF445082|+|2788-3343|ARO:3002577|AAC(6')-Ib4,555,0,555,+,7593,2524,87,642,553,555,255,NM:i:2,100.000000
5,gb|AF043381|+|251-863|ARO:3002580|AAC(6')-Ib9,612,25,612,+,7593,2524,55,642,585,587,255,NM:i:2,95.915033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,gb|AJ318094|+|0-861|ARO:3000961|TEM-94,861,0,861,+,5967,1044,36,897,852,861,255,NM:i:9,100.000000
376,gb|AJ308558|+|181-1042|ARO:3000962|TEM-95,861,0,861,+,5967,1044,36,897,859,861,255,NM:i:2,100.000000
377,gb|AY092401|+|0-861|ARO:3000963|TEM-96,861,0,861,+,5967,1044,36,897,858,861,255,NM:i:3,100.000000
378,gb|AM990992.1|-|1001760-1003680|ARO:3000186|tetM,1920,0,1920,+,1107,47514,33707,35627,1874,1920,255,NM:i:46,100.000000


In [33]:
#Get the specific columns from GraphAligner
GraphAligner_Combined_Query_Path=GraphAligner_Combined1[['Query','Path Matching', 'Start Position on Path', 
'End Position on Path']]
# Convert the integer to string
GraphAligner_Combined_Query_Path['Query']=GraphAligner_Combined_Query_Path['Query'].astype(str)
GraphAligner_Combined_Query_Path['Path Matching']=GraphAligner_Combined_Query_Path['Path Matching'].astype(str)
GraphAligner_Combined_Query_Path['Start Position on Path']=GraphAligner_Combined_Query_Path['Start Position on Path'].astype(str)
GraphAligner_Combined_Query_Path['End Position on Path']=GraphAligner_Combined_Query_Path['End Position on Path'].astype(str)



/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1467/2646241093.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GraphAligner_Combined_Query_Path['Query']=GraphAligner_Combined_Query_Path['Query'].astype(str)
/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/ipykernel_1467/2646241093.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GraphAligner_Combined_Query_Path['Path Matching']=GraphAligner_Combined_Query_Path['Path Matching'].astype(str)
/var/folders/j4/d0_t23bn3yv8zw6bmbhbzvfw0000gn/T/i

In [34]:
#Use merge operation so that we have the sequences for both softwares for successful queries
merged_df_Graph = GraphAligner_Combined_Query_Path.groupby('Query').agg({'Path Matching':' '.join, 'Start Position on Path': ', '.join,'End Position on Path': ', '.join}).reset_index()
merged_df_Graph

,Query,Path Matching,Start Position on Path,End Position on Path
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,5151,23,884
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,5967,36,897
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,5151,23,884
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,5151,23,884
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,5151,23,884
...,...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,5967,36,897
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,5967,36,894
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,5967,36,894
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,5967,36,894


In [35]:
merged_df_Graph.rename(columns = {'Path Matching':'Path_GraphAligner'}, inplace = True)
merged_df_Graph.rename(columns = {'Start Position on Path':'Start_GraphAligner'}, inplace = True)
merged_df_Graph.rename(columns = {'End Position on Path':'End_GraphAligner'}, inplace = True)

In [36]:
merged_df_Graph

,Query,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,5151,23,884
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,5967,36,897
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,5151,23,884
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,5151,23,884
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,5151,23,884
...,...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,5967,36,897
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,5967,36,894
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,5967,36,894
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,5967,36,894


## Compile the Queries and the results from GraphAligner in one dataframe

In [37]:
#Use merge operation so that we have the sequences for both softwares for successful queries
df_QueryAndResultsGraphAligner=pd.merge(df,merged_df_Graph, on='Query',how="left")
df_QueryAndResultsGraphAligner

,Query,Sequence,Length,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,5151,23,884
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,5967,36,897
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,5151,23,884
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,5151,23,884
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,5151,23,884
...,...,...,...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,5967,36,897
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,5967,36,894
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,5967,36,894
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,5967,36,894


## Compile all the results in one master Dataframe

In [38]:
merged_df_1 = pd.merge(df_QueryAndResultsBandage, df_QueryAndResultsSPAligner, on='Query', how='inner')
merged_df_master = pd.merge(merged_df_1, df_QueryAndResultsGraphAligner, on='Query', how='inner')
merged_df_master

,Query,Sequence_x,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Sequence_Bandage,Sequence_y,Length_x,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Sequence,Length_y,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,NaN,NaN,NaN,NaN,NaN,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,5151,23,884
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,36.0,897.0,861.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,5967,36,897
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,23.0,884.0,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,5151,23,884
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,23.0,884.0,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,5151,23,884
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,23.0,884.0,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,5151,23,884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,36.0,897.0,861.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,5967,36,897
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,36.0,894.0,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,5967,36,894
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,36.0,894.0,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,5967,36,894
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,36.0,894.0,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,5967,36,894


In [42]:
#Drop some columns

merged_df_master.drop(['Sequence_y', 'Length_x','Length_y','Sequence'], axis=1)


,Query,Sequence_x,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Sequence_Bandage,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,NaN,NaN,NaN,NaN,NaN,5151,23,884
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,36.0,897.0,861.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,36,897
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,23.0,884.0,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,5151,23,884
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,23.0,884.0,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,5151,23,884
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,23.0,884.0,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,5151,23,884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,36.0,897.0,861.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,36,897
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,36.0,894.0,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,36,894
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,36.0,894.0,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,36,894
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,36.0,894.0,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,5967,36,894


In [41]:
merged_df_master

,Query,Sequence_x,Length_Bandage,Path_Bandage,Start_Bandage,End_Bandage,Sequence_Bandage,Sequence_y,Length_x,Start_SPAligner,End_SPAligner,Length_SPAligner,Path_SPAligner,Sequence_SPAligner,Sequence,Length_y,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,NaN,NaN,NaN,NaN,NaN,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,5151,23,884
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,36.0,897.0,861.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,5967,36,897
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,23.0,884.0,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,5151,23,884
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,23.0,884.0,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,5151,23,884
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,(24) 5151- (884),24,884,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,23.0,884.0,861.0,5151-,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,ATGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGC...,861,5151,23,884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,gb|Y14574|+|0-861|ARO:3000888|TEM-17,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,(37) 5967+ (897),37,897,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,36.0,897.0,861.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,861,5967,36,897
374,gb|Y17581|+|78-936|ARO:3000891|TEM-20,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,36.0,894.0,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,5967,36,894
375,gb|Y17582|+|0-858|ARO:3000892|TEM-21,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,36.0,894.0,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,5967,36,894
376,gb|Y17583|+|213-1071|ARO:3000893|TEM-22,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,(37) 5967+ (894),37,894,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,36.0,894.0,858.0,5967+,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,ATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGG...,858,5967,36,894


## Save the Queries and  all the Results  in one .tsv file


In [ ]:
merged_df_master.to_csv("QueriesAndResultsMasterList.tsv",sep='\t')

## This section saves the Nodes and Sequences from the gfa file

In [ ]:
import gfapy

# Open the GFA file
file_path = "graph1.gfa"
gfa = gfapy.Gfa.from_file(file_path)

num_segments = len(gfa.segments)

In [ ]:
data_graph = []

In [ ]:
# Store the name and sequence for each node from the graph
for segment in gfa.segments:
    data_graph.append({"Name": segment.name, "Sequence": segment.sequence})

# Convert the list to a DataFrame
df_graph = pd.DataFrame(data_graph)

# Print the DataFrame
print(df_graph)

In [ ]:
#Count the number of duplicates
#len(df_graph['Name'])-len(df_graph['Name'].drop_duplicates())

In [ ]:
num_segments

In [ ]:
#Save only one copy of each row to the dataframe
df_graph=df_graph.drop_duplicates()

In [ ]:
df_graph

In [ ]:
df_graph['Name']=df_graph['Name'].astype(str)


## Get the sequences unique to GraphAligner
## Get the dataframe containing the path from GraphAligner


In [ ]:
df_GraphAligner_Paths= pd.read_csv('GraphAlignerNotEqual.tsv', sep='\t')
#GraphAlignerNotEqual.tsv is from another script:GraphAlignerBandageScript.ipynb

In [ ]:
df_GraphAligner_Paths

In [ ]:
df_GraphAligner_Paths
df_GraphAligner_Paths.sort_index(inplace=True)
df_GraphAligner_Paths['Path Matching']=df_GraphAligner_Paths['Path Matching'].astype(str)

In [ ]:
# Function to process the Path values
def process_path(path):
    path = path.replace('<', ' ').replace('>', ' ')
    path = ' '.join(path.split('<'))
    return path

In [ ]:
df_GraphAligner_Paths['Path Matching']=df_GraphAligner_Paths['Path Matching'].apply(process_path).str.strip()


In [ ]:
df_GraphAligner_Paths
# Create a new column 'Combined_Sequence' that concatenates sequences if Path has two values


In [ ]:
df_GraphAligner_Paths[['Path1', 'Path2']] = df_GraphAligner_Paths['Path Matching'].str.split(' ', expand=True)

In [ ]:
df_GraphAligner_Paths

In [ ]:
# Get the rows where Column2 is not None
rows_not_none = df_GraphAligner_Paths[df_GraphAligner_Paths['Path2'].notna()]

# Print the resulting dataframe
rows_not_none

In [ ]:
#Get the sequence for Path 1 and Path 2
# Merge df1 and df2 based on matching values in Column1 and Name

merged_df= pd.merge(df_GraphAligner_Paths, df_graph, left_on='Path1', right_on='Name')
merged_df = pd.merge(merged_df, df_graph, left_on='Path2', right_on='Name')

# Concatenate the sequences
merged_df

In [ ]:
merged_df = merged_df[['Sequence_x'][:len()-1]]